In [1]:
import cv2
import numpy as np
import pandas as pd
from keras import backend as k
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from tqdm import tqdm

/Users/kids/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Intial importing of data

In [2]:
#  Get working directory and
WORKING_DIR = "./data/"
# Location of labels
LABELS = WORKING_DIR + "labels.csv"
# Example of the submission text
TEST = WORKING_DIR + "sample_submission.csv"

# Location of train and test folders
TRAIN_FOLDER = WORKING_DIR + "/train/"
TEST_FOLDER = WORKING_DIR + "/test/"

# Read in the labels and the test data
df_train = pd.read_csv(LABELS)
df_test = pd.read_csv(TEST)

targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse=True)
one_hot_labels = np.asarray(one_hot)


In [3]:
i = 0 
im_size = 224 # Max size is 299
num_output =120
x_train_0 = []
y_train_0 = []



for f, breed in tqdm(df_train.values):
    img = cv2.imread('{}{}.jpg'.format(TRAIN_FOLDER,f))
    label = one_hot_labels[i]
    x_train_0.append(cv2.resize(img, (im_size, im_size)))
    y_train_0.append(label)
    i += 1


100%|██████████| 10222/10222 [00:51<00:00, 198.41it/s]


In [4]:
# Resizing and retraining test dat
x_test_0 = []
for f in tqdm(df_test['id'].values):
    img = cv2.imread('{}{}.jpg'.format(TEST_FOLDER,f))
    x_test_0.append(cv2.resize(img, (im_size, im_size)))

100%|██████████| 10357/10357 [00:52<00:00, 197.33it/s]


In [5]:
#  Converting to 32 and from 0 to 1

y_train_raw = np.array(y_train_0, np.uint8)
x_train_raw = np.array(x_train_0, np.float32) / 255.
x_test  = np.array(x_test_0, np.float32) / 255.

# Check shape
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

#


(10222, 224, 224, 3)
(10222, 120)
(10357, 224, 224, 3)


In [6]:
# Splitting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(x_train_raw, y_train_raw, test_size=0.2, random_state=1)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)



((8177, 224, 224, 3), (8177, 120), (2045, 224, 224, 3), (2045, 120))


In [7]:

# Implement Batch Normalization

k.set_image_dim_ordering('tf')

# fix random seed for reproducibility
seed = 123
np.random.seed(seed)
# Number of classes

# This is if I would like to define my own CNN

In [8]:
def multilayer_cnn_model():
    # YOUR TURN
    # Build a model with 4 convolutional layers
    # choose your own hyperparameters for conv layers
    # choose to include maxpool if you like
    # choose to include dropout if you like
    # create model
    inner_model = Sequential()

    # Add 32 filters
    inner_model.add(Conv2D(64, kernel_size=3, padding='same',
                           input_shape=(im_size, im_size, 3)))
    inner_model.add(Activation('relu'))

    # Conv2D 32 3 x3
    inner_model.add(Conv2D(64, kernel_size=3, padding='same'))
    inner_model.add(Activation('relu'))
    # 2x2 pooling
    inner_model.add(MaxPooling2D((2, 2)))
    # Conv2D 64  3x3
    inner_model.add(Conv2D(64, kernel_size=3, padding='same'))
    inner_model.add(Activation('relu'))
    # Conv 3D 8x8
    inner_model.add(Conv2D(64, kernel_size=3, padding='same'))
    inner_model.add(Activation('relu'))
    # 2x2 pooling
    inner_model.add(MaxPooling2D((2, 2)))
    # Flatten
    inner_model.add(Flatten())

    # . Density layer
    inner_model.add(Dense(512, activation='relu'))
    # . Density layer
    inner_model.add(Dense(512, activation='relu'))
    #     # output layer
    inner_model.add(Dense(num_output, activation='softmax'))
    inner_model.summary()
    # Compile model using the same options
    inner_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return inner_model

In [9]:
def bn_model():
    # YOUR TURN
    # Create a model with 4 convolutional layers (2 repeating VGG stype units) and 2 dense layers before the output
    # Use Batch Normalization for every conv and dense layers
    # Use dropout layers if you like
    # Use Adam optimizer

    model = Sequential()
    model.add(Conv2D(16, 3, input_shape=(im_size, im_size, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(16, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(num_output))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [10]:

# build the model

# trained_model = "multilayer"  # multilayer or bn_model
trained_model = "multilayer"
if trained_model == "multilayer":
    model = multilayer_cnn_model()
elif trained_model == "bn_model":
    model = bn_model()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
activation_2 (Activation)    (None, 224, 224, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 64)      36928     
_________________________________________________________________
activation_3 (Activation)    (None, 112, 112, 64)      0         
__________

In [11]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200,
                    verbose=2)  # Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100 - scores[1] * 100))



Train on 8177 samples, validate on 2045 samples
Epoch 1/20
 - 2312s - loss: 5.0958 - acc: 0.0088 - val_loss: 4.7856 - val_acc: 0.0088
Epoch 2/20
 - 2197s - loss: 4.7822 - acc: 0.0113 - val_loss: 4.7822 - val_acc: 0.0088
Epoch 3/20
 - 2170s - loss: 4.7826 - acc: 0.0120 - val_loss: 4.7808 - val_acc: 0.0108
Epoch 4/20
 - 2161s - loss: 4.7736 - acc: 0.0117 - val_loss: 4.7767 - val_acc: 0.0093
Epoch 5/20
 - 2163s - loss: 4.7114 - acc: 0.0215 - val_loss: 4.6834 - val_acc: 0.0284
Epoch 6/20
 - 2167s - loss: 4.1926 - acc: 0.1028 - val_loss: 4.9246 - val_acc: 0.0303
Epoch 7/20
 - 2186s - loss: 2.3993 - acc: 0.4388 - val_loss: 6.7260 - val_acc: 0.0328
Epoch 8/20
 - 2205s - loss: 0.6860 - acc: 0.8395 - val_loss: 9.6178 - val_acc: 0.0337
Epoch 9/20
 - 2209s - loss: 0.2592 - acc: 0.9450 - val_loss: 10.9439 - val_acc: 0.0308
Epoch 10/20
 - 2212s - loss: 0.1562 - acc: 0.9688 - val_loss: 11.3113 - val_acc: 0.0254
Epoch 11/20
 - 2215s - loss: 0.0965 - acc: 0.9810 - val_loss: 11.9167 - val_acc: 0.0313
E

In [12]:
save_model = False
if save_model:
# serialize model to JSON
    model_json = model.to_json()
    with open("{}/{}_model.json".format(WORKING_DIR,trained_model), "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("{}/{}_model.h5".format(WORKING_DIR,trained_model))
    print("Saved model to disk")


In [13]:

preds = model.predict(x_test, verbose=1)

sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.head(5)
# Saving results
sub.to_csv("{}/results_{}.csv".format(WORKING_DIR, trained_model), mode='w', index=False)



10357/10357 [==============================] - 937s 90ms/step


# This is for running against previous defined models

In [14]:
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

Below the user can select from a few different models that have allready been used.  More information at the website https://keras.io/applications/

In [15]:
# trained_model = "vgg16"  # inception, vgg16, resnet50, mobilenet
# trained_model = "inception"
# trained_model = "resnet50"
trained_model = "mobilenet"

# Idea from https://keras.io/applications/
if trained_model == "inception":
    # create the base pre-trained model
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

elif trained_model == "vgg16":
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

elif trained_model == "resnet50":
    base_model =  ResNet50(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

elif trained_model == "mobilenet":
    base_model = MobileNet(input_shape=(im_size, im_size, 3), include_top=False, weights='imagenet')

else:
    sys.exit(" Could not find the train model to start with")
# add a global spatial average pooling layer

In [16]:
# Here we start with the base model
x = base_model.output
# Then we add pooling 2d and a FCC layer
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer with the final guess
predictions = Dense(num_output, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
# Code idea came from https://gist.github.com/Hironsan/e041d6606164bc14c50aa56b989c5fc0
# Function came from https://gist.github.com/Hironsan/e041d6606164bc14c50aa56b989c5fc0
def batch_iter(data, labels, batch_size_def, shuffle=False):
    num_batches_per_epoch = int((len(data) - 1) / batch_size_def) + 1

    def data_generator():
        data_size = len(data)
        while True:
            # Shuffle the data at each epoch
            if shuffle:
                shuffle_indices = np.random.permutation(np.arange(data_size))
                shuffled_data = data[shuffle_indices]
                shuffled_labels = labels[shuffle_indices]
            else:
                shuffled_data = data
                shuffled_labels = labels

            for batch_num in range(num_batches_per_epoch):
                start_index = batch_num * batch_size_def
                end_index = min((batch_num + 1) * batch_size_def, data_size)
                x_value, y_value = shuffled_data[start_index: end_index], shuffled_labels[start_index: end_index]
                yield x_value, y_value

    return num_batches_per_epoch, data_generator()




In [24]:
import time

#. Here we only need a few epochs because we started with a trained model
batch_size = 32
num_epochs = 30

#. Select different train and valid steps for each batch to switch things up
train_steps, train_batches = batch_iter(X_train, y_train, batch_size)
valid_steps, valid_batches = batch_iter(X_test, y_test, batch_size)
# Want to figure out the time of the different methods
t0=time.time()
model.fit_generator(train_batches, train_steps, epochs=num_epochs, validation_data=valid_batches,
                    validation_steps=valid_steps)
t1=time.time()
print(t1-t0," seconds")



Epoch 1/30
256/256 [==============================] - 660s 3s/step - loss: 0.1134 - acc: 0.9775 - val_loss: 2.1735 - val_acc: 0.5164
Epoch 2/30
256/256 [==============================] - 675s 3s/step - loss: 0.0825 - acc: 0.9857 - val_loss: 2.3492 - val_acc: 0.5032
Epoch 3/30
256/256 [==============================] - 677s 3s/step - loss: 0.0722 - acc: 0.9841 - val_loss: 2.1394 - val_acc: 0.5340
Epoch 4/30
256/256 [==============================] - 684s 3s/step - loss: 0.0653 - acc: 0.9878 - val_loss: 2.2761 - val_acc: 0.5394
Epoch 5/30
256/256 [==============================] - 653s 3s/step - loss: 0.0692 - acc: 0.9835 - val_loss: 2.3547 - val_acc: 0.5350
Epoch 6/30
256/256 [==============================] - 635s 2s/step - loss: 0.0658 - acc: 0.9840 - val_loss: 2.7690 - val_acc: 0.5022
Epoch 7/30
256/256 [==============================] - 632s 2s/step - loss: 0.0608 - acc: 0.9852 - val_loss: 2.5448 - val_acc: 0.5159
Epoch 8/30
256/256 [==============================] - 630s 2s/step - 

The different methods used here were:
* Vgg16 :  Got score of 1.56173 on kaggle
* Inception: Got a score of  
* resnet50:  Got a score of 5.8588 on kaggle.  Seemed to be overfitting the problem
* mobilenet


In [25]:
preds = model.predict(x_test, verbose=1)

10357/10357 [==============================] - 699s 67ms/step


In [23]:
sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.head(5)
# Saving results
sub.to_csv("{}/results_{}.csv".format(WORKING_DIR, trained_model), mode='w', index=False)

Upload the data to https://www.kaggle.com/c/dog-breed-identification/submit